In [1]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy as nltk_accuracy
from collections import defaultdict
import random

In [2]:
# Descargar recursos necesarios
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/daniyes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/daniyes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
path_data = "/mnt/c/Users/dyesi/Documents/Platzi/Algoritmos_clasificacion_texto/BD_HIST_PRG.xlsx"
df = pd.read_excel(path_data)


In [4]:
list_tuplas = list(zip(df['Actividad'], df['Sistema']))

In [5]:
# Función para preprocesar el texto
def preprocesar(texto):
    stop_words = set(stopwords.words('spanish'))
    tokens = word_tokenize(texto.lower())
    tokens_filtrados = [word for word in tokens if word.isalnum() and word not in stop_words]
    return tokens_filtrados

# Preparar los datos para el clasificador
datos_preprocesados = [(preprocesar(texto), categoria) for (texto, categoria) in list_tuplas]

In [6]:
freq_dist_por_sistema = defaultdict(nltk.FreqDist)
for texto, sistema in datos_preprocesados:
    freq_dist_por_sistema[sistema].update(texto)


In [7]:
total_tokens_por_sistema = {
    sistema: sum(fd.values()) for sistema, fd in freq_dist_por_sistema.items()
}

porcentaje_freq_dist_por_sistema = {
    sistema: {token: freq / total_tokens_por_sistema[sistema] if freq / total_tokens_por_sistema[sistema] >= 0.00009 else 0
              for token, freq in fd.items()}
    for sistema, fd in freq_dist_por_sistema.items()}

In [8]:
def calcular_atributos(frase, sistemas, porcentaje_freq_dist_por_sistema):
    atributos = {}
    for sistema in sistemas:
        atributos[sistema] = sum(
            porcentaje_freq_dist_por_sistema[sistema].get(token, 0)
            for token in frase
        )
    return atributos

# Aplicar a los datos
sistemas = list(freq_dist_por_sistema.keys())
caracteristicas = [
    (calcular_atributos(texto, sistemas, porcentaje_freq_dist_por_sistema), sistema)
    for texto, sistema in datos_preprocesados]

random.shuffle(caracteristicas)

In [9]:
# Dividir en entrenamiento y prueba
tamanio_entrenamiento = int(0.8 * len(caracteristicas))
entrenamiento = caracteristicas[:tamanio_entrenamiento]
prueba = caracteristicas[tamanio_entrenamiento:]

# Entrenar el clasificador
clasificador = NaiveBayesClassifier.train(entrenamiento)

# Evaluar el clasificador
precision = nltk.classify.accuracy(clasificador, prueba)
print(f"Precisión del clasificador: {precision:.2f}")


Precisión del clasificador: 0.90


In [10]:
novedad = "parachoques posterior costado derecho golpeado"
novedad_tokens = preprocesar(novedad)

atributos_novedad = calcular_atributos(novedad_tokens, sistemas, porcentaje_freq_dist_por_sistema)

# Clasificar
sistema_predicho = clasificador.classify(atributos_novedad)
print(f"El sistema vehicular predicho es: {sistema_predicho}")

El sistema vehicular predicho es: CR-GENERAL


In [11]:
atributos_novedad

{'CH-BATERIA ALTA': 0.0005957700327673518,
 'CH-DIRECCION': 0.0002403846153846154,
 'CR-GENERAL': 0.029868412768368695,
 'CH-DIFERENCIAL': 0,
 'CH-SUSPENSION': 0.0023580271173118494,
 'LL-LLANTAS': 0,
 'CH-FRENOS': 0.0005935598753524262,
 'GR-VEHICULO': 0.00018574805808848363,
 'CH-NEUMATICO': 0,
 'IT-ITS': 0,
 'CR-LATONERIA': 0.20887643875295372,
 'CH-BASTIDOR': 0,
 'CH-ELECTRONICO': 0.001992914083259522,
 'GR-LAVADO': 0,
 'CH-BATERIA BAJA': 0.008771929824561403,
 nan: 0.019736842105263157,
 'CH-REFRIGERACION': 0.0036496350364963502,
 'IT-SIRCI': 0}

In [12]:
class_counts = df['Sistema'].value_counts()
class_counts


Sistema
GR-VEHICULO         19860
CR-GENERAL          17697
LL-LLANTAS          13053
CR-LATONERIA         6911
IT-ITS               4349
CH-FRENOS            1394
GR-LAVADO            1323
CH-ELECTRONICO       1122
CH-NEUMATICO         1064
CH-SUSPENSION         951
CH-DIRECCION          901
CH-BATERIA ALTA       843
CH-DIFERENCIAL        348
CH-BATERIA BAJA       143
CH-REFRIGERACION       68
IT-SIRCI                8
CH-BASTIDOR             1
Name: count, dtype: int64

: 

In [ ]:
from imblearn.over_sampling import SMOTE
import pandas as pd

# Supongamos que df es tu dataframe original
# X es el conjunto de características
# y es la columna de las etiquetas (e.g., 'Sistema')
df_sample = df.sample(1000, random_state=42)


X = df.drop('Sistema', axis=1)
y = df['Sistema']

# Convertir las variables categóricas en variables dummy
X = pd.get_dummies(X, drop_first=True)

# Aplicar SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Convertir el DataFrame balanceado a un dataset resampled
df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
df_resampled['Sistema'] = y_resampled